In [1]:
import pandas as pd
import numpy as np

# Import Dataset

In [3]:
dataset_xls_1 = pd.read_excel('./Dataset/sales-(2020).xlsx')
dataset_xls_2 = pd.read_excel('./Dataset/sales-(2021).xlsx')
dataset_xls_3 = pd.read_excel('./Dataset/sales-(2022).xlsx')
dataset_xls_4 = pd.read_excel('./Dataset/sales-(2023).xlsx')

In [4]:
# combine all dataset_xls into one dataframe 
dataset = pd.concat([dataset_xls_1, dataset_xls_2, dataset_xls_3, dataset_xls_4], ignore_index=True)

In [4]:
dataset.columns

Index(['username', 'no-pesanan', 'tanggal-pesanan', 'nama-barang', 'jumlah',
       'dibayar', 'payment-method'],
      dtype='object')

In [5]:
dataset

username    no-pesanan tanggal-pesanan   
0         surya251196   201104NURT3      2020-05-03  \
1     captain.marline   201104DSB34      2020-05-03   
2       ridhayashinta  201104JHVJ45      2020-05-03   
3    nurulismawati103   201104VGRX4      2020-05-03   
4       nabilahauraaa   201104XSAEF      2020-05-04   
..                ...           ...             ...   
395        kidnafis94   210504ARK78      2021-05-08   
396    kris_thsuteleo   210504AGT67      2021-05-08   
397       nenahruzain   210504JNHP0      2021-05-08   
398       erwin121088   210504FPN12      2021-05-08   
399        aang_tirgo   210504MYLP8      2021-05-08   

                               nama-barang  jumlah  dibayar payment-method  
0         Gamis Hiitam, Gamis Dress Wanita       2   120000            COD  
1               Nibras Couple, Gamis Hitam       2   180000            COD  
2                  Koko Kurta, Gamis Hitam       2   120000            COD  
3                       Gamis Dress Wanita       1    45000            COD  
4    Gamis Wanita Polos, Gamis Dress Hitam       2   120000      Pay Later  
..                                     ...     ...      ...            ...  
395                           Gamis Hitam        1    45000            COD  
396                           Gamis Hitam        1    45000            COD  
397                           Gamis Hitam        1    45000            COD  
398                           Gamis Hitam        1    45000            COD  
399                           Gamis Hitam        1    45000            COD  

[400 rows x 7 columns]

## Tokenize nama-barang

In [6]:
dataset['nama-barang'] = dataset['nama-barang'].str.split(', ')
dataset['nama-barang'] = dataset['nama-barang'].apply(
    lambda x: [item.strip() for item in x])

In [7]:
dataset.head()

username    no-pesanan tanggal-pesanan   
0       surya251196   201104NURT3      2020-05-03  \
1   captain.marline   201104DSB34      2020-05-03   
2     ridhayashinta  201104JHVJ45      2020-05-03   
3  nurulismawati103   201104VGRX4      2020-05-03   
4     nabilahauraaa   201104XSAEF      2020-05-04   

                               nama-barang  jumlah  dibayar payment-method  
0       [Gamis Hiitam, Gamis Dress Wanita]       2   120000            COD  
1             [Nibras Couple, Gamis Hitam]       2   180000            COD  
2                [Koko Kurta, Gamis Hitam]       2   120000            COD  
3                     [Gamis Dress Wanita]       1    45000            COD  
4  [Gamis Wanita Polos, Gamis Dress Hitam]       2   120000      Pay Later

# Implement Apriori Algorithm

In [95]:
from apyori import apriori

In [96]:
result = list(apriori(dataset['nama-barang']))
result

[RelationRecord(items=frozenset({'Gamis Dress Wanita'}), support=0.1975, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gamis Dress Wanita'}), confidence=0.1975, lift=1.0)]),
 RelationRecord(items=frozenset({'Gamis Hiitam'}), support=0.1025, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gamis Hiitam'}), confidence=0.1025, lift=1.0)]),
 RelationRecord(items=frozenset({'Gamis Hitam'}), support=0.1525, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gamis Hitam'}), confidence=0.1525, lift=1.0)]),
 RelationRecord(items=frozenset({'Gamis Wanita Polos'}), support=0.1625, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gamis Wanita Polos'}), confidence=0.1625, lift=1.0)]),
 RelationRecord(items=frozenset({'Koko Hitam'}), support=0.35, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Koko Hitam'}), confidence=0.35, lift

# Implement FP Growth Algorithm

In [8]:
from itertools import groupby
from collections import Counter


In [9]:
# join all items in each transaction into one array
transactions = dataset['nama-barang'].tolist()
# append each transaction into one array
transaction_items = []
for transaction in transactions:
    for item in transaction:
        transaction_items.append(item)


### __Frekuensi Kemunculan tiap item

In [10]:
items_df = pd.DataFrame(transaction_items, columns=["items"])
items_df["incident_count"] = 1

items_df_table = items_df.groupby("items").sum().sort_values(
    "incident_count", ascending=False).reset_index()

items_df_table


items  incident_count
0          Koko Hitam             140
1       Nibras Couple             117
2          Koko Kurta             101
3  Gamis Dress Wanita              79
4  Gamis Wanita Polos              65
5         Gamis Hitam              61
6        Gamis Hiitam              41
7   Gamis Dress Hitam               5

### __Plotting with Treemap

In [11]:
import plotly.express as px


In [12]:
items_df_table["all"] = "Top 50 items"

fig_items = px.treemap(items_df_table.head(50), path=['all', "items"], values='incident_count',
                       color=items_df_table["incident_count"].head(50), hover_data=['items'],
                       color_continuous_scale='Blues',
                       )


In [13]:
fig_items.show()

### __Pre-Processing Dataset

In [14]:
# crate new datarames that each column is unique item and each row is number of transaction
items_transaction_df = pd.DataFrame(columns=items_df_table["items"].tolist())

In [15]:

# fill each row with items in dataset['nama-barang] and fill with 1 if item is in transaction and 0 if not
for transaction in dataset['nama-barang']:
    temp_df = pd.DataFrame(columns=items_df_table["items"].tolist())
    for item in items_df_table["items"].tolist():
        if item in transaction:
            temp_df[item] = [True]
        else:
            temp_df[item] = [False]
    items_transaction_df = pd.concat(
        [items_transaction_df, temp_df], ignore_index=True)

items_transaction_df


Koko Hitam Nibras Couple Koko Kurta Gamis Dress Wanita Gamis Wanita Polos   
0        False         False      False               True              False  \
1        False          True      False              False              False   
2        False         False       True              False              False   
3        False         False      False               True              False   
4        False         False      False              False               True   
..         ...           ...        ...                ...                ...   
395      False         False      False              False              False   
396      False         False      False              False              False   
397      False         False      False              False              False   
398      False         False      False              False              False   
399      False         False      False              False              False   

    Gamis Hitam Gamis Hiitam Gamis Dress Hitam  
0         False         True             False  
1          True        False             False  
2          True        False             False  
3         False        False             False  
4         False        False              True  
..          ...          ...               ...  
395        True        False             False  
396        True        False             False  
397        True        False             False  
398        True        False             False  
399        True        False             False  

[400 rows x 8 columns]

### __Implementing FP-Growth Algorithm

In [16]:
from mlxtend.frequent_patterns import fpgrowth, association_rules


In [23]:
fpgrowth_res = fpgrowth(items_transaction_df, min_support=0.05, use_colnames=True)
fpgrowth_res 

/home/lp212/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



support                          itemsets
0   0.1975              (Gamis Dress Wanita)
1   0.1025                    (Gamis Hiitam)
2   0.2925                   (Nibras Couple)
3   0.1525                     (Gamis Hitam)
4   0.2525                      (Koko Kurta)
5   0.1625              (Gamis Wanita Polos)
6   0.3500                      (Koko Hitam)
7   0.0575  (Koko Hitam, Gamis Dress Wanita)
8   0.0600       (Koko Hitam, Nibras Couple)
9   0.0550          (Koko Hitam, Koko Kurta)

In [24]:
association_rules_res = association_rules(
    fpgrowth_res, metric="confidence", min_threshold=1)
association_rules_res


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []

In [20]:
# Sort values based on confidence
association_rules_res.sort_values("confidence", ascending=False)


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []